In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [62]:
class IterativeBase:
    def __init__(self, symbol, start, end, amount, use_spread = True):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.amount = amount
        self.position = 0
        self.initial_balance = amount
        self.current_balance = amount
        self.units = 0
        self.trades = 0
        self.use_spread = use_spread #eza bdna trading cost inc
        self.get_data()

    def get_data(self):
        raw = yf.download(self.symbol, start=self.start, end=self.end)
        raw.reset_index(inplace=True)
        raw['Spread'] = raw['High'] - raw['Low']
        raw.rename(columns={"Date": "time", "Close": "price"}, inplace=True)  # Rename first
        raw = raw[['time', 'price', 'Spread']].copy()  # Then subset
        raw = raw.loc[(raw['time'] >= self.start) & (raw['time'] <= self.end)]
        raw["returns"] = np.log(raw["price"] / raw["price"].shift(1))
        self.data = raw

    def plot_data(self, cols=None):
        if cols is None:
            cols = "price"
        self.data[cols].plot(figsize=(12, 8), title=self.symbol)
        plt.show()

    def get_values(self, bar):
        date = str(self.data['time'].iloc[bar].date())
        price = round(self.data['price'].iloc[bar], 5)
        spread = round(self.data['Spread'].iloc[bar], 5)
        return date, price, spread
    def print_current_position_value(self,bar):
        date,price,spread = self.get_values(bar)
        cpv = self.units * price
        print("{} | current Position Value = {} ".format(date,round(cpv,2)))

    def print_current_balance(self, bar):
        date, price, spread = self.get_values(bar)
        print(f"{date} | Current balance: {self.current_balance:.2f}")

    def buy_instrument(self, bar, units=None, amount=None):
        date, price, spread = self.get_values(bar)
        if self.use_spread:
            price += spread/2 #ask price 
        if amount is not None:
            units = int(amount / price)
        if units * price > self.current_balance:
            print("Insufficient funds to complete the purchase.")
            return
        self.current_balance -= units * price
        self.units += units
        self.trades += 1
        print(f"{date} | Buying {units} units for {price:.2f}")
    def sell_instrument(self,bar,units = None , amount = None):
        date, price, spread = self.get_values(bar)
        if self.use_spread:
            price -= spread/2 #3m nemche negative bi sel
        if amount is not None:
            units = int(amount / price)
        if units * price > self.current_balance:
            print("Insufficient funds to complete the purchase.")
            return
        self.current_balance += units * price
        self.units -= units
        self.trades += 1
        print(f"{date} | Selling {units} units for {price:.2f}")

    def __repr__(self):
        return f"IterativeBase(symbol={self.symbol}, start={self.start}, end={self.end}, current_balance={self.current_balance}, units={self.units}, trades={self.trades})"
    def close_pos(self,bar):
        date,price,spread = self.get_values(bar)
        print(75 * "-")
        print("{} | +++ CLOSING FINAL POISTION +++".format(date))
        self.current_balance += self.units*price #if it was a sell units ha tkoun - so balance will decrease
        self.current_balance -= (abs(self.units)*spread/2*self.use_spread)#7*False =0
        print("{} | closing position of {}  for {} ".format(date,self.units,price))
        self.units = 0
        self.trades += 1
        perf = (self.current_balance - self.initial_balance) / self.initial_balance * 100
        self.print_current_balance(bar)
        print("{} | net perfomamnce (%) = {}".format(date,round(perf,2)))
        print("{} | number of trades executedd = {} ".format(date,self.trades))
        print(75 * "-")
        

In [65]:
class IterativeBacktest(IterativeBase):
    def __init__(self, symbol, start, end, amount, use_spread=True):
        super().__init__(symbol, start, end, amount, use_spread)
        self.position = 0

    def go_long(self, bar, units=None, amount=None):
        if self.position == -1:  # If currently in a short position
            self.buy_instrument(bar, units=self.units)  # Close short position by buying
        if units:
            self.buy_instrument(bar, units=units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.buy_instrument(bar, amount=amount)
        self.position = 1

    def go_short(self, bar, units=None, amount=None):
        if self.position == 1:  # If currently in a long position
            self.sell_instrument(bar, units=self.units)  # Close long position by selling
        if units:
            self.sell_instrument(bar, units=units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.sell_instrument(bar, amount=amount)
        self.position = -1

    def test_sma_strategy(self, SMA_S, SMA_L):
        self.position = 0
        self.trades = 0
        self.current_balance = self.initial_balance
        self.get_data()
        self.data["SMA_S"] = self.data["price"].rolling(window=SMA_S).mean()
        self.data["SMA_L"] = self.data["price"].rolling(window=SMA_L).mean()

        for bar in range(len(self.data) - 1):
            if self.data["SMA_S"].iloc[bar] > self.data["SMA_L"].iloc[bar]:
                if self.position in [0, -1]:
                    self.go_long(bar, amount="all")
            elif self.data["SMA_S"].iloc[bar] < self.data["SMA_L"].iloc[bar]:
                if self.position in [0, 1]:
                    self.go_short(bar, amount="all")
            if self.position != 0:
                self.close_pos(len(self.data) - 1)

In [55]:
k.buy_instrument(0,units= 8000000) #supose you want to buy at day 1 (bar  = 0) 10k units 

2023-01-01 | Buying 8000000 units for 16679.68


In [57]:
k.print_current_balance(0) #we lost from trading cost

2023-01-01 | Current balance: 9866562539040.00


In [58]:
k.print_current_position_value(0)

2023-01-01 | current Position Value = 133000640640.0 


In [59]:
#supose we holded for the position for the last year ( after 1 year here)
k.print_current_balance(-1) #not change mosriyetk ma da2et fioyn

2024-05-04 | Current balance: 9866562539040.00


In [60]:
k.print_current_position_value(-1)

2024-05-04 | current Position Value = 511131781280.0 


In [61]:
k.close_pos(-1) 

---------------------------------------------------------------------------
2024-05-04 | +++ CLOSING FINAL POISTION +++
2024-05-04 | closing position of 8000000  for 63891.47266 
2024-05-04 | Current balance: 10370111898440.00
2024-05-04 | net perfomamnce (%) = 3.7
2024-05-04 | number of trades executedd = 2 
---------------------------------------------------------------------------
